<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/files_diffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from openpyxl import styles
import pandas as pd
import numpy as np
from pathlib import Path
import openpyxl
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule


def excel_diff(path_OLD, path_NEW, index_col, bill_cols,path_save, type_file='csv'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    bill_cols = lower_str(bill_cols)

    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD, encoding='latin').fillna('')
        old_cols = lower_str(list(df_OLD.columns))
        df_OLD.columns = old_cols
        df_OLD = df_OLD[bill_cols]
        df_OLD['site'] = df_OLD[index_col]
        df_OLD = df_OLD.set_index('site')

        df_NEW = pd.read_excel(path_NEW, encoding='latin').fillna('')
        new_cols = lower_str(list(df_NEW.columns))
        df_NEW.columns = new_cols
        df_NEW = df_NEW[bill_cols]
        df_NEW['site'] = df_NEW[index_col]
        df_NEW = df_NEW.set_index('site')
    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        old_cols = lower_str(list(df_OLD.columns))
        df_OLD.columns = old_cols
        df_OLD = df_OLD[bill_cols]
        df_OLD['site'] = df_OLD[index_col]
        df_OLD = df_OLD.set_index('site')

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        new_cols = lower_str(list(df_NEW.columns))
        df_NEW.columns = new_cols
        df_NEW = df_NEW[bill_cols]
        df_NEW['site'] = df_NEW[index_col]
        df_NEW = df_NEW.set_index('site')
    
    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                    #new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)
    new_copy = new_copy.dropna(how='all', axis=0)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])

    new_copy = new_copy.sort_index().fillna('')
    #print(new_copy)
    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get xlsxwriter objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    #ws = wb['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
    
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    new_copy = new_copy.reset_index()
    new_copy = new_copy[bill_cols]
    for site in new_copy[index_col]:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')



In [16]:
hu_bill_cols = ["Active / not active",\
    "Confirmed Skylon scope",\
    "Code",\
    "Categorization by Transmission Sys",\
    "Additional information \x96 room configuration (including container information)",\
    "Active Sharing Arrangements involving the Operator",\
    "Critical site",\
    "Strategic site",\
    "Telenor tenant",\
    "Telekom tenant",\
    "Power supply for tenants",\
    "Energy category",\
    'Ready For Active Installation ("RFAI") date',\
    "Wip_Site",\
    "Bts_Site",\
    "Strategic_Site_Bucket",\
    "Critical_Site_Beyond_10",\
    "Subsequent_Sharing_Arrangement",\
    "First_Active_Sharing_Start_Date",\
    "First_Active_Sharing_End_Date",\
    "Active Sharing Deployment Types",\
    "Sites_As_Metered_Estimated",\
    "Date of equipment removal"]
path_old = '/content/TowerDB_Hungary_20210630.csv'
path_new = '/content/TowerDB_Hungary_20210731.csv'
to_save = '/content/'
#(path_OLD, path_NEW, index_col, bill_cols, type_file='csv', dt_type=2, number_format=2)
excel_diff(path_old, path_new, 'code', hu_bill_cols, to_save)


New Rows:     ['S0113']
Dropped Rows: []

Done.

